In [238]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
import warnings
from matplotlib.lines import Line2D
import scipy.stats
import matplotlib.ticker as mtick
warnings.filterwarnings('ignore')
#pd.set_option("display.max_columns", None)
from IPython.display import HTML

In [239]:
df = pd.read_csv("RPT-8430_CompiledData_Final.csv")

In [240]:
# pd.set_option("display.max_columns", None)
# df.head(5)

In [241]:
dfm = df.copy()

In [242]:
dfm = dfm[dfm.ProcessingStep == "Raw"]
dfm = dfm[dfm.Channel == "Yellow_530_555"]


dfm = dfm[(dfm.SampleID.str[:6] == "HPVneg") |
          (dfm.SampleID.str[:5] == "Bglob") |
          (dfm.SampleID.str[:3] == "Neg") |
          (dfm.SampleID.str[:3] == "ZAS") |
          (dfm.SampleID.str[:2] == "NC") |
          (dfm.SampleID.str[:5] == "HPV16") |
          (dfm.SampleID.str[:5] == "SiHav") |
          (dfm.SampleID.str[:5] == "HeLav") |
          (dfm.SampleID.str[:4] == "A549") |
          (dfm.SampleID.str[:4] == "16hv") |
          (dfm.SampleID.str[:4] == "Sure") |
          (dfm.SampleID.str[:5] == "37Neg") |
          (dfm.SampleID.str[:5] == "HPV18") ]

total = dfm.shape[0]


dfm.filter(items= ["SampleUId","SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])
print('Total_presumed_negatives')
print(dfm.shape[0])
#print("{0:.2%}".format(dfm.shape[0]/total))


dffp = dfm[dfm.LocalizedResult == "TargetAmplified"]
print('Number_of_False_Positives')
print(dffp.shape[0])
print('False_Positives_Rate')
print("{0:.2%}".format(dffp.shape[0]/total))
dfm = df.copy()

Total_presumed_negatives
1572
Number_of_False_Positives
33
False_Positives_Rate
2.10%


In [243]:
dfc = dfm.copy()

In [244]:
dfc = dfc[dfc.ProcessingStep == "Raw"]
dfc = dfc[dfc.Channel == "Yellow_530_555"]
dfc = dfc[(dfc.SampleID.str[:6] == "HPVneg") |
          (dfc.SampleID.str[:5] == "Bglob") |
          (dfc.SampleID.str[:3] == "Neg") |
          (dfc.SampleID.str[:3] == "ZAS") |
          (dfc.SampleID.str[:2] == "NC") |
          (dfc.SampleID.str[:5] == "HPV16") |
          (dfc.SampleID.str[:5] == "SiHav") |
          (dfc.SampleID.str[:5] == "HeLav") |
          (dfc.SampleID.str[:4] == "16hv") |
          (dfc.SampleID.str[:4] == "Sure") |
          (dfc.SampleID.str[:5] == "37Neg") |
          (dfc.SampleID.str[:4] == "A549") |
          (dfc.SampleID.str[:5] == "HPV18") ]
conditions = [
    (dfc['BaselineFirstCycle'] == 10.0) & (dfc['BaselineLastCycle'] == 16.0),
    (dfc['BaselineFirstCycle'] == 10.0) & (dfc['BaselineLastCycle'] == 10.0),
    (dfc['BaselineFirstCycle'] != 10.0) & (dfc['MaxPeakHeight'] > 0),
    (dfc['BaselineFirstCycle'] != 10.0)
    ]

# create a list of the values we want to assign for each condition
values = ['Fixed_Baseline_Sample', 'Single_Point_Normalized_Sample', 'Weak_Amplification_Samples', 'Unassigned']

# create a new column and use np.select to assign values to it using our lists as arguments
dfc['Condition'] = np.select(conditions, values)

dfc = dfc.filter(items= ["SampleUId","Condition",'BaselineFirstCycle','BaselineLastCycle',"SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])

# display updated DataFrame



In [245]:
dfw = dfc[dfc.Condition == 'Weak_Amplification_Samples']
dfw = dfw.filter(items= ["SampleUId","Condition",'BaselineFirstCycle','BaselineLastCycle',"SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])
print('Total_Weak_Amplification_Samples')
print(dfw.shape[0])

dfwfp = dfw[dfw.LocalizedResult == "TargetAmplified"]
print('Number_of_False_Positives_in_Weak_Amplification_Samples')
print(dfwfp.shape[0])
print('False_Positives_Rate_in_Weak_Amplification_Samples')
print("{0:.2%}".format(dfwfp.shape[0]/dfw.shape[0]))

Total_Weak_Amplification_Samples
93
Number_of_False_Positives_in_Weak_Amplification_Samples
32
False_Positives_Rate_in_Weak_Amplification_Samples
34.41%


In [246]:
dff = dfc[dfc.Condition == 'Fixed_Baseline_Sample']
dff = dff.filter(items= ["SampleUId","Condition",'BaselineFirstCycle','BaselineLastCycle',"SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])


print('Total_Fixed_Baseline_Samples')
print(dff.shape[0])
dfffp = dff[dff.LocalizedResult == "TargetAmplified"]
print('Number_of_False_Positives_in_Fixed_Baseline_Samples')
print(dfffp.shape[0])
print('False_Positives_Rate_in_Fixed_Baseline_Samples')
print("{0:.2%}".format(dfffp.shape[0]/dff.shape[0]))


Total_Fixed_Baseline_Samples
29
Number_of_False_Positives_in_Fixed_Baseline_Samples
0
False_Positives_Rate_in_Fixed_Baseline_Samples
0.00%


In [247]:
dfs = dfc[dfc.Condition == 'Single_Point_Normalized_Sample']
dfs = dfs.filter(items= ["SampleUId","Condition",'BaselineFirstCycle','BaselineLastCycle',"SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])
print('Total_Single_Point_Normalized_Samples')
print(dfs.shape[0])
dfsfp = dfs[dfs.LocalizedResult == "TargetAmplified"]
print('Number_of_False_Positives_in_Single_Point_Normalized_Samples')
print(dfsfp.shape[0])
print('False_Positives_Rate_in_Single_Point_Normalized_Samples')
print("{0:.2%}".format(dfsfp.shape[0]/dfs.shape[0]))

Total_Single_Point_Normalized_Samples
8
Number_of_False_Positives_in_Single_Point_Normalized_Samples
1
False_Positives_Rate_in_Single_Point_Normalized_Samples
12.50%


In [248]:
dfa = dfc[dfc.Condition == 'Unassigned']
dfa = dfa.filter(items= ["SampleUId","Condition",'BaselineFirstCycle','BaselineLastCycle',"SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])
print('Total_Unassigned')
print(dfa.shape[0])
dfafp = dfa[dfa.LocalizedResult == "TargetAmplified"]
print('Number_of_False_Positives_in_Unassigned')
print(dfafp.shape[0])
print('False_Positives_Rate_in_Unassigned')
print("{0:.2%}".format(dfafp.shape[0]/dfa.shape[0]))

Total_Unassigned
1442
Number_of_False_Positives_in_Unassigned
0
False_Positives_Rate_in_Unassigned
0.00%


In [249]:

fp_summary = pd.DataFrame({'Fixed_Baseline_Sample': [dfffp.shape[0], "{0:.2%}".format(dfffp.shape[0]/dff.shape[0]), dff.shape[0]], 
                           'Single_Point_Normalized_Sample': [dfsfp.shape[0], "{0:.2%}".format(dfsfp.shape[0]/dfs.shape[0]), dfs.shape[0]],
                           'Weak_Amplification_Samples': [dfwfp.shape[0], "{0:.2%}".format(dfwfp.shape[0]/dfw.shape[0]), dfw.shape[0]],
                           'Unassigned': [dfafp.shape[0], "{0:.2%}".format(dfafp.shape[0]/dfa.shape[0]), dfa.shape[0]],
                           'Total': ["{0:.2%}".format(dffp.shape[0]/total), dffp.shape[0], total]},
                            index=['Number_of_False_Positvie', 'Rate_of_False_Positvie', 'Total'])




fp_summary

,Fixed_Baseline_Sample,Single_Point_Normalized_Sample,Weak_Amplification_Samples,Unassigned,Total
Number_of_False_Positvie,0,1,32,0,2.10%
Rate_of_False_Positvie,0.00%,12.50%,34.41%,0.00%,33
Total,29,8,93,1442,1572
